# Spark pools demo

In [1]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, udf, lit

StatementMeta(large, 6, 1, Finished, Available)

#### Spark Session
*spark* is the spark session, that provides access to Spark resources
*df* is a Spark Dataframe. 

In [2]:
df = spark.read.load('abfss://data@paoltadls.dfs.core.windows.net/open-datasets/weatherAUS.csv', format='csv', header=True)

StatementMeta(small, 3, 3, Finished, Available)

In [3]:
display(df.limit(10))

StatementMeta(small, 3, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 034fcb5a-d941-4266-8c1c-ebf58a4c243e)

In [6]:
df.count()

StatementMeta(small, 3, 7, Finished, Available)

145460

In [30]:
df.describe

StatementMeta(small, 3, 31, Finished, Available)

<bound method DataFrame.describe of DataFrame[Date: string, Location: string, MinTemp: string, MaxTemp: string, Rainfall: string, Evaporation: string, Sunshine: string, WindGustDir: string, WindGustSpeed: string, WindDir9am: string, WindDir3pm: string, WindSpeed9am: string, WindSpeed3pm: string, Humidity9am: string, Humidity3pm: string, Pressure9am: string, Pressure3pm: string, Cloud9am: string, Cloud3pm: string, Temp9am: string, Temp3pm: string, RainToday: string, RainTomorrow: string]>

#### Lazy vs concrete operations
The following statement is lazy.
It is actully executed when we display it (or write etc.).

Note that reading a CSV file as above is not lazy, and data is actually loaded to memory, as CSV has not metadata available to describe the data structure

In [28]:
df_agg = df \
.withColumn("MaxTemp",df.MaxTemp.cast(FloatType())) \
.withColumn("MinTemp",df.MinTemp.cast(FloatType())) \
.select(["Location", "MinTemp", "MaxTemp"]) \
.groupBy("Location") \
. avg()


StatementMeta(small, 3, 29, Finished, Available)

In [29]:
display(df_agg)

StatementMeta(small, 3, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 09673454-5d76-45de-a4e0-b73a150e39b9)

In [ ]:
import pyspark.sql.functions as F

df = df.withColumn("MinMaxTemp", F.udf(lambda x, y: f(x,y))("MinTemp", "MaxTemp"))
display(df)

#### Data distribution

*df* is a distributed object - it is stored to cluster nodes memory

In [31]:
df.rdd.getNumPartitions()

StatementMeta(small, 3, 32, Finished, Available)

4

In [32]:
df.repartition(8).rdd.getNumPartitions()

StatementMeta(small, 3, 33, Finished, Available)

8

Depending on the use case, we might be willing to repartition by a certain columns, e.g., to optimize some join/where statements. 

In [36]:
df=df.repartition('Location')

StatementMeta(small, 3, 37, Finished, Available)

In [37]:
df.rdd.getNumPartitions()

StatementMeta(small, 3, 38, Finished, Available)

200

The *cache()* statement keeps that repartitioned df in memory... remember repartitioning is a heavy operstion that we usually perform depending on the specific use case.

In [45]:
df.cache()

StatementMeta(small, 3, 46, Finished, Available)

DataFrame[Date: string, MinTemp: string, MaxTemp: string, Rainfall: string, Evaporation: string, Sunshine: string, WindGustDir: string, WindGustSpeed: string, WindDir9am: string, WindDir3pm: string, WindSpeed9am: string, WindSpeed3pm: string, Humidity9am: string, Humidity3pm: string, Pressure9am: string, Pressure3pm: string, Cloud9am: string, Cloud3pm: string, Temp9am: string, Temp3pm: string, RainToday: string, RainTomorrow: string, Location: string]

We can specify a partitioning column when writing to disk. It doesn't necessary be the same as the one we are using for the dataframe... that one depends on use cases!

In [42]:
df.write \
.partitionBy('Location') \
.mode('overwrite') \
.parquet('abfss://data@paoltadls.dfs.core.windows.net/open-datasets/weatherPAR')

StatementMeta(small, 3, 43, Finished, Available)

In [43]:
df = spark.read.load('abfss://data@paoltadls.dfs.core.windows.net/open-datasets/weatherPAR')

StatementMeta(small, 3, 44, Finished, Available)

Note when re-reading the number of partitions is higher (there are 200 partitions on disk, however, the number of partitions in memory depends on the number of actual workers).

In [44]:
df.rdd.getNumPartitions()

StatementMeta(small, 3, 45, Finished, Available)

7

It is easy to transform Spark Dataframes to **Pandas Dataframes**, however, they will run in the **application driver** memory space!
On the contrary, Spark Dataframes run in the cluster's memory (i.e., they are distributed objects).

In [9]:
pdf = df.limit(100).toPandas()

StatementMeta(small, 3, 10, Finished, Available)

In [46]:
pdf.rdd.getNumPartitions()

StatementMeta(small, 3, 47, Finished, Available)

AttributeError: 'DataFrame' object has no attribute 'rdd'

In [ ]:
#### Using UDF
UDF provides a lot of flexibility...

In [75]:
def avgTemp (temp1, temp2) -> float:
    try:
        temp1 = float(temp1)
        temp2 = float(temp2)
        return (temp1+temp2)/2.0

    except:
        return None

print(avgTemp(2.1,'4.8'))
print(avgTemp(2.1,"four dot eight"))

StatementMeta(small, 3, 76, Finished, Available)

3.45
None

In [77]:
avgTempUDF = udf(lambda x,y:avgTemp(x,y),FloatType())   

df \
.withColumn("AvgTemp", avgTempUDF(col("MinTemp"), col("MaxTemp"))) \
.select(["Location", "MinTemp", "MaxTemp", "AvgTemp"]) \
.show(truncate=False)

StatementMeta(small, 3, 78, Finished, Available)

+--------+-------+-------+-------+
|Location|MinTemp|MaxTemp|AvgTemp|
+--------+-------+-------+-------+
|Canberra|8      |24.3   |16.15  |
|Canberra|14     |26.9   |20.45  |
|Canberra|13.7   |23.4   |18.55  |
|Canberra|13.3   |15.5   |14.4   |
|Canberra|7.6    |16.1   |11.85  |
|Canberra|6.2    |16.9   |11.55  |
|Canberra|6.1    |18.2   |12.15  |
|Canberra|8.3    |17     |12.65  |
|Canberra|8.8    |19.5   |14.15  |
|Canberra|8.4    |22.8   |15.6   |
|Canberra|9.1    |25.2   |17.15  |
|Canberra|8.5    |27.3   |17.9   |
|Canberra|10.1   |27.9   |19.0   |
|Canberra|12.1   |30.9   |21.5   |
|Canberra|10.1   |31.2   |20.65  |
|Canberra|12.4   |32.1   |22.25  |
|Canberra|13.8   |31.2   |22.5   |
|Canberra|11.7   |30     |20.85  |
|Canberra|12.4   |32.3   |22.35  |
|Canberra|15.6   |33.4   |24.5   |
+--------+-------+-------+-------+
only showing top 20 rows

However.... native functions provide better performance!

In [86]:
df = df \
.withColumn("MaxTemp",df.MaxTemp.cast(FloatType())) \
.withColumn("MinTemp",df.MinTemp.cast(FloatType())) \

df.withColumn("AvgTemp", (df["MinTemp"] + df["MaxTemp"])/lit(2.0)) \
.select(["Location", "MinTemp", "MaxTemp", "AvgTemp"]) \
.show()


StatementMeta(small, 3, 87, Finished, Available)

+--------+-------+-------+------------------+
|Location|MinTemp|MaxTemp|           AvgTemp|
+--------+-------+-------+------------------+
|Canberra|    8.0|   24.3|16.149999618530273|
|Canberra|   14.0|   26.9|20.450000762939453|
|Canberra|   13.7|   23.4|18.549999237060547|
|Canberra|   13.3|   15.5|14.399999618530273|
|Canberra|    7.6|   16.1|11.850000381469727|
|Canberra|    6.2|   16.9|11.549999237060547|
|Canberra|    6.1|   18.2| 12.15000057220459|
|Canberra|    8.3|   17.0|12.649999618530273|
|Canberra|    8.8|   19.5|14.149999618530273|
|Canberra|    8.4|   22.8| 15.59999942779541|
|Canberra|    9.1|   25.2|17.150001525878906|
|Canberra|    8.5|   27.3|17.899999618530273|
|Canberra|   10.1|   27.9|              19.0|
|Canberra|   12.1|   30.9|              21.5|
|Canberra|   10.1|   31.2|20.650001525878906|
|Canberra|   12.4|   32.1|             22.25|
|Canberra|   13.8|   31.2|              22.5|
|Canberra|   11.7|   30.0|20.850000381469727|
|Canberra|   12.4|   32.3|22.34999